In [1]:
import random

import numpy as np


In [2]:
with open('High/High Grass.obj') as f:
    data = f.read().strip().split('\n')


vertices = np.array([[float(j) for j in i[2:].split(' ')] for i in data if i.startswith('v ')])
faces = [i for i in data if i.startswith('f ')]
texture_vertices = np.array([[float(j) for j in i[3:].split(' ')] for i in data if i.startswith('vt ')])

# fine clusters
v_clusters = []
for face in faces:
    face_vs = [int(f.split('/')[0]) for f in face.strip().split()[1:]]
    v_cluster_index = None
    for candind_index, candid_cluster in enumerate(v_clusters):
        if any([v in candid_cluster for v in face_vs]):
            v_cluster_index = candind_index
            break
    if v_cluster_index is not None:
        v_clusters[v_cluster_index].update(set(face_vs))
    else:
        v_clusters.append(set(face_vs))

print(len(v_clusters))


163


In [3]:
# coarse clusters


def join_clusters(v_clusters: list[set]) -> list[set]:
    coarse_clusters = []
    for query_cluster in v_clusters:
        ref_cluster_index = None
        for candind_index, candid_cluster in enumerate(coarse_clusters):
            if any([v in candid_cluster for v in query_cluster]):
                ref_cluster_index = candind_index
                break
        if ref_cluster_index is not None:
            coarse_clusters[ref_cluster_index].update(query_cluster)
        else:
            coarse_clusters.append(query_cluster)
    return coarse_clusters


for i in range(10):
    v_clusters = join_clusters(v_clusters)
print(len(v_clusters))


40


In [8]:
for cluster_index, v_cluster in enumerate(v_clusters):
    v_cluster_indices = np.array(sorted(v_cluster)) - 1
    vs = vertices[v_cluster_indices]
    with open(f'/tmp/zg{cluster_index:03d}.obj', 'w') as f:
        # vertices
        f.write('v ' + '\nv '.join([' '.join([str(i) for i in v]) for v in vs]) + '\n')
        # texture vertices
        for i in range(1, len(vertices) + 1):
            f.write(f'vt {random.random()} {random.random()} 0\n')
        # faces
        face_counter = 0
        for face in faces:
            vs = [int(i.split('/')[0]) for i in face.strip()[2:].split(' ')]
            if any([v in v_cluster for v in vs]):
                new_face = 'f ' + ' '.join([str(j) + '/' + str(face_counter + i) for i, j in enumerate(vs)]) + '\n'
                f.write(new_face)
            face_counter += len(vs)


In [6]:
random.random()


0.3113731336321167